# Title

<a id='intro'></a>
## Introduction

> **Research has shown that the increase of food consumption can be due to coping mechanisms for stress or other mental health issues and has risen with the increase of technology in daily life.** 

The purpose of this study is 

### Contents:
<ul>
    <li><a href="#intro">Introduction</a></li>
    <li><a href="#source">Data Source</a></li>
    <li><a href="#wrangle">Data Wrangling & Cleaning</a></li>
    <li><a href="#explore">Exploratory Data Analysis</a></li>
        <ul>
        <li><a href="#Q1">Q1. How has the consumption of food per person (per day) changed over time throughout the world?</a></li>
        <li><a href="#Q2">Q2. Is consumption of food per person related by region of the world?</a></li>
        <li><a href="#Q3">Q3. Is there a correlation between the food consumption per person in a country with that country's rate of suicide?</a></li>
        <li><a href="#Q4">Q4. Is there a correlation between the food consumption per person in a country with the number of internet users in that country?</a></li>
        </ul>
    <li><a href="#summary">Summary & Conclusions</a></li>
</ul>

<a id='source'></a>
## Data Sources

Access the README.md documentation regarding the specific source of the data used in this analysis. 

All the data was retrived through [Gapminder](http://www.gapminder.org). Gapminder is an independent Swedish foundation with no political, religious or economic affiliations.


<a id='wrangle'></a>
## Data Wrangling & Cleaning

Before starting to analyze the data, the quality of the datasets downloaded from the Gapminder database is evaluated and then cleaned appropriately. First, various libaries are imported that will be used at different steps of this process and later with visualizations.   

This is link to country_converter: https://github.com/konstantinstadler/country_converter

In [285]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from functools import reduce
import pycountry # connects country names with country codes
import country_converter as coco # connects country code with its continent 

%matplotlib inline

#### Wrangling *population* and *foodsupply* raw datasets
As hinted by the inital questions stated above, knowing the population numbers will be helpful to normalize to compare different datasets approprately. 

Since the purpose of this quanantative correlation study wants to explore relationships (if any) between the different data sets, it is important that there are consistent year ranges. First, the world population dataset is explored to understand its general properties.

In [286]:
# read population csv
df_population = pd.read_csv('population_total.csv')

# view structure of dataset
df_population.head(3)

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,3280000,3280000,3280000,3280000,3280000,3280000,3280000,3280000,3280000,...,76600000,76400000,76300000,76100000,76000000,75800000,75600000,75400000,75200000,74900000
1,Albania,400000,402000,404000,405000,407000,409000,411000,413000,414000,...,1330000,1300000,1270000,1250000,1220000,1190000,1170000,1140000,1110000,1090000
2,Algeria,2500000,2510000,2520000,2530000,2540000,2550000,2560000,2560000,2570000,...,70400000,70500000,70500000,70600000,70700000,70700000,70700000,70700000,70700000,70700000


In [287]:
# understand the shape of population dataset
df_population.shape

(195, 302)

The dataset on the consumption of food is primary to answering the questions drafted for this project. This is the next dataset explored after global population.

In [288]:
# read food consumption csv
df_foodsupply = pd.read_csv('food_supply_kilocalories_per_person_and_day.csv')

# view structure of dataset
df_foodsupply.head(3)

,country,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,3000.0,2920.0,2700.0,2950.0,2960.0,2740.0,2970.0,2920.0,2940.0,...,1970.0,1950.0,1970,2050,2040,2080,2100,2110,2100,2090
1,Albania,2220.0,2240.0,2160.0,2270.0,2250.0,2250.0,2260.0,2340.0,2400.0,...,2790.0,2870.0,2860,2860,2950,2990,3080,3130,3180,3190
2,Algeria,1620.0,1570.0,1530.0,1540.0,1590.0,1570.0,1650.0,1710.0,1710.0,...,2990.0,2960.0,3050,3040,3050,3110,3140,3220,3270,3300


In [289]:
# understand the shape of the foodsupply dataset
df_foodsupply.shape

(168, 54)

The population dataset year range is much broader than the foodsupply. (Population has 301 years of data while Food Supply has 53.) It is important that both datasets have the same year ranges since comparisons will be made. Therefore the years out of the range of the foodsupply dataset will be removed from the population dataset.

When exploring the dataset it was noted that there is data on future years in the population dataset. This raises a flag on the integrity of the data. Since the project is limited to Gapminder data - these columns will be deleted so the last column is 2013, similar to the foodsupply dataset.

In [290]:
# remove years in population 
df_population.drop(df_population.loc[:,'1800':'1960'].columns, axis=1, inplace=True)
df_population.drop(df_population.loc[:,'2014':'2100'].columns, axis=1, inplace=True)

# confirm columns in both datasets match
(df_population.columns == df_foodsupply.columns).all()

True

#### Wrangling *suicide* and *internet* raw datasets
The factor of number of suicide in each country as a measure of mental health and the number of internet users per country as a measure of the rise of technology in daily life, will be datasets used for analysis later on in conjunction with the population and food supply datasets. 

The previous actions done to population and foodsupply will be done with the suicide and internet data. 

In [291]:
# read suicide csv
df_suicide = pd.read_csv('suicide_total_deaths.csv')

# view structure of dataset
df_suicide.head(3)

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Afghanistan,703.0,754.0,820.0,894.0,977.0,1050.0,1100.0,1130.0,1170.0,...,1680.0,1710.0,1750.0,1760.0,1810.0,1870.0,1990.0,2080.0,2170.0,2250.0
1,Albania,127.0,130.0,131.0,135.0,136.0,142.0,150.0,162.0,170.0,...,204.0,205.0,201.0,195.0,191.0,188.0,186.0,184.0,183.0,181.0
2,Algeria,806.0,822.0,843.0,866.0,888.0,912.0,941.0,983.0,1020.0,...,1240.0,1250.0,1270.0,1290.0,1310.0,1340.0,1370.0,1410.0,1420.0,1440.0


The year ranges for the population and foodsupply were 1961 - 2013.  Ideally, these other two measures will need comparable year ranges. Since foodsupply will be used compare to each of these, columns will be dropped to match to ensure foodsupply, internet users, and suicide numbers will show only the years of 1990-2013. 

Since the <a href="#Q1">first question</a> of the analysis only looks at population and food supply a seperate merged table will be created that gives an expanded set of years.

In [292]:
# read internet csv
df_internet = pd.read_csv('it_net_user_zs.csv')

# view structure of dataset
df_internet.head(3)

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,5.45,5.9,7.0,8.26,11.2,13.5,13.5,13.5
1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,49.0,54.70,57.2,60.1,63.30,66.4,71.8,71.8,69.6
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.5,14.9,18.20,22.5,29.5,38.20,42.9,47.7,59.6,59.6


At first glance above there might be several null values with the early years of Internet user table. The null values within the chart will be inspected more closely.    

In [293]:
# exploring null values for years in internet users
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 61 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  194 non-null    object 
 1   1960     7 non-null      float64
 2   1961     0 non-null      float64
 3   1962     0 non-null      float64
 4   1963     0 non-null      float64
 5   1964     0 non-null      float64
 6   1965     7 non-null      float64
 7   1966     0 non-null      float64
 8   1967     0 non-null      float64
 9   1968     0 non-null      float64
 10  1969     0 non-null      float64
 11  1970     7 non-null      float64
 12  1971     0 non-null      float64
 13  1972     0 non-null      float64
 14  1973     0 non-null      float64
 15  1974     0 non-null      float64
 16  1975     7 non-null      float64
 17  1976     7 non-null      float64
 18  1977     7 non-null      float64
 19  1978     7 non-null      float64
 20  1979     7 non-null      float64
 21  1980     7 non-n

There are several null values 1960-1989. Keeping the range of 1990 - 2013 which matches with the numbers of suicides will work well for analysis. Both datasets will have years dropped outside of the range of 1990-2013. 

In [294]:
# remove unneeded year columns in internet dataset
df_internet.drop(df_internet.loc[:,'1960':'1989'].columns, axis=1, inplace=True)
df_internet.drop(df_internet.loc[:,'2014':'2019'].columns, axis=1, inplace=True)

# remove unneeded year columns in suicide dataset
df_suicide.drop(df_suicide.loc[:,'2014':'2016'].columns, axis=1, inplace=True)

# confirm columns in both datasets match
(df_internet.columns == df_suicide.columns).all()

True

#### Dropping Unique Countries
All 4 datasets will have all the same countries among all the datasets. 

In [296]:
def country_diff_details(df1, df2):
    list1 = df1.country.unique()
    list2 = df2.country.unique()
    
    # countries to be deleted from df1
    list3 = list(set(list1)-set(list2))
    
    # countries to be deleted from df2
    list4 = list(set(list2)-set(list1))
    
    # both datasets differ by these countries
    list5 = list3 + list4
    numb_list5 = len(list5)
    
    print("There will be ", numb_list5 ," countries dropped between the datasets.\n")
    print("Those countries will include:", list5)
    
    return

In [297]:
country_diff_details(df_population, df_foodsupply)

There will be  27  countries dropped between the datasets.

Those countries will include: ['Qatar', 'Tuvalu', 'Burundi', 'Bhutan', 'Papua New Guinea', 'Congo, Dem. Rep.', 'Marshall Islands', 'Seychelles', 'Micronesia, Fed. Sts.', 'Singapore', 'Liechtenstein', 'Palestine', 'South Sudan', 'Palau', 'Bahrain', 'Syria', 'Comoros', 'San Marino', 'Monaco', 'Andorra', 'Equatorial Guinea', 'Eritrea', 'Holy See', 'Libya', 'Nauru', 'Tonga', 'Somalia']


In [298]:
def country_differences(df1, df2):
    """
    Args:
        (dataframe) df1: dataframe 1 designed by user
        (dataframe) df2: dataframe 2 designed by user
    Return:
        (dataframe) df1: dataframe 1 with removed rows (as necessary)
        (dataframe) df2: dataframe 2 with removed rows (as necessary)

    Finds the countries not contained in both datasets and drops those countries from each so they both match.
    """
    list1 = df1.country.unique()
    list2 = df2.country.unique()
    
    # countries to be deleted from df1
    list3 = list(set(list1)-set(list2))
    
    # countries to be deleted from df2
    list4 = list(set(list2)-set(list1))
        
    df1new = df1[~df1['country'].isin(list3)]
    df2new = df2[~df2['country'].isin(list4)]

    return df1new, df2new

In [299]:
df_new_population, df_new_foodsupply = country_differences(df_population, df_foodsupply)

In [300]:
# confirm countries match 
country_diff_details(df_new_population, df_new_foodsupply)

There will be  0  countries dropped between the datasets.

Those countries will include: []


In [311]:
# showing the final number of countries in dataset
df_new_population.shape

(168, 54)

Although the columns of years allow good opportunity for comparison, creating a 'year' column with multiple listings for each country will lend better to visualizations and aggregations later in the project. Therefore the [melt panda](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function will be used so that each year will be an individual row and sorted by country and year. 

A function is written below that will be able to be applied to other dataframes in the future. The function will take two or more dataframes, individually melt it and then merge. After merging, it sorts by year and country. This will be used for faciliate the process for future dataframes as well since all data is gathered from Gapminder share similar formats.

In [301]:
def df_melt(dfs, column_name):
    """
    Uses panda melt arrange Gapminder data in better format; sorts by year and country. 
    
    Args:
        (dataframe) dfs: list of dataframes
        (str) volumn_name: value name for column
    
    Returns:
        (dataframe) df_melt : dataframe containing the merged df formatted for analysis
    """
    
    # list created
    df_new = []
    
    # loop over list in df
    for i, x in enumerate(dfs):
        df_melt = pd.melt(dfs[i], id_vars = ["country"], var_name = 'year', value_name = column_name[i])
        df_melt.sort_values(["country", "year"], inplace = True)
    
        # append to created list
        df_new.append(df_melt)
    
    return df_new

In [302]:
dataframes = [df_new_population, df_new_foodsupply]
column_names = ["population", "foodsupply"]

df_popfood = df_melt(dataframes, column_names)
df_newfood = df_popfood[1]
df_newfood.head()

,country,year,foodsupply
0,Afghanistan,1961,3000.0
168,Afghanistan,1962,2920.0
336,Afghanistan,1963,2700.0
504,Afghanistan,1964,2950.0
672,Afghanistan,1965,2960.0


In [303]:
df_newpop = df_popfood[0]
df_newpop.head()

,country,year,population
0,Afghanistan,1961,9170000
168,Afghanistan,1962,9350000
336,Afghanistan,1963,9540000
504,Afghanistan,1964,9740000
672,Afghanistan,1965,9960000


In [304]:
# reduces the melted dataframes into one
df_popfood_final = reduce(lambda left,right: pd.merge(left,right, on=["country", "year"], how = 'left'), df_popfood)

In [305]:
# confirm
df_popfood_final.head()

,country,year,population,foodsupply
0,Afghanistan,1961,9170000,3000.0
1,Afghanistan,1962,9350000,2920.0
2,Afghanistan,1963,9540000,2700.0
3,Afghanistan,1964,9740000,2950.0
4,Afghanistan,1965,9960000,2960.0


In [306]:
#take care of null values
df_popfood_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8904 entries, 0 to 8903
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     8904 non-null   object 
 1   year        8904 non-null   object 
 2   population  8904 non-null   int64  
 3   foodsupply  8022 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 347.8+ KB


In [212]:
# attempting to locate the null values in foodsupply

# looking at value counts of nulls across the years
df_popfood.dropna(subset=["foodsupply"]).year.value_counts()

2011    168
2007    168
2012    168
2013    168
2010    168
2006    168
2009    168
2008    168
2004    166
2002    166
2003    166
2001    166
2000    166
2005    166
1995    164
1994    164
1996    164
1999    164
1993    164
1998    164
1997    164
1992    161
1990    142
1991    142
1970    141
1965    141
1980    141
1973    141
1974    141
1985    141
1977    141
1976    141
1979    141
1969    141
1986    141
1988    141
1967    141
1966    141
1989    141
1983    141
1961    141
1987    141
1981    141
1963    141
1984    141
1971    141
1968    141
1978    141
1964    141
1972    141
1962    141
1982    141
1975    141
Name: year, dtype: int64

Created function below that documents that number of missing values per country. 

In [307]:
def missing_bycountry(df, column):
    """
    Args:
        (dataframe) df = dataframe
        (str) column = column
    Return:
        prints country list (str) with number of missing values (int)
    """
    missing = []
    for x in list((df["country"]).unique()):
        missing_n = sum(df[df["country"] == x][column].isnull())
        
        if missing_n > 0:
            missing.append(x)
            print (x, "-", missing_n)
    
    return missing    

In [266]:
missing_bycountry(df_popfood[1], "foodsupply");

Armenia - 31
Azerbaijan - 31
Belarus - 31
Belgium - 39
Bosnia and Herzegovina - 31
Croatia - 31
Czech Republic - 32
Estonia - 31
Ethiopia - 32
Georgia - 31
Kazakhstan - 31
Kyrgyz Republic - 31
Latvia - 31
Lithuania - 31
Luxembourg - 39
Moldova - 31
Montenegro - 45
North Macedonia - 31
Oman - 29
Russia - 31
Serbia - 45
Slovak Republic - 32
Slovenia - 31
Tajikistan - 31
Turkmenistan - 31
Ukraine - 31
Uzbekistan - 31



There are 27 countries that have missing data between the years of 1990 - 2013. Expected that some countries have not data recorded for the earlier years. 

18 countries have 31 null values, which is the minimum except for Oman with 29. Leaving 7 countries with more missing years. These 7 countries will be deleted from the dataset.

For the countries with missing data, surveying shows that the 31 null values are the 31 early years. Since the analysis for later work will focus on 1990 - 2013, the values for 1990 and 1991 will be estimated and filled in. 

In [314]:
df_popfood_final.query("country == 'Russia'")

,country,year,population,foodsupply
6572,Russia,1961,121000000,NaN
6573,Russia,1962,123000000,NaN
6574,Russia,1963,124000000,NaN
6575,Russia,1964,125000000,NaN
6576,Russia,1965,127000000,NaN
6577,Russia,1966,127000000,NaN
6578,Russia,1967,128000000,NaN
6579,Russia,1968,129000000,NaN
6580,Russia,1969,129000000,NaN
6581,Russia,1970,130000000,NaN


In [315]:
# drop countries determined above
country_drop = ["Belgium", "Czech Republic", "Ethiopia", "Luxembourg", "Montenegro", "Serbia", "Slovak Republic"]
df_popfood_final_v2 = df_popfood_final[~df_popfood_final['country'].isin(country_drop)]

# check if countries are dropped by using Serbia as an example
df_popfood_final_v2.query("country == 'Serbia'")

,country,year,population,foodsupply


Two data points will be estimated for those missing data from 1990 and 1991. This will allow the countries to be included for later analysis when the suicide figures and internet users are considered. A copy of the dataframe is taken now however to save a dataframe for analysis allowing more nations to be considered when comparing with population. 

However the countries that will require fill in data will also need to be eliminated from this dataset for analysis. 

In [317]:
# copy dataframe for food consumption and population analysis
df_popfood_v3 = df_popfood_final_v2.copy()

#drop the countries that will require fill in data
countries_fillin_data = ["Armenia", "Azerbaijan", "Belarus", "Bosnia and Herzegovina", "Croatia", "Estonia", "Georgia", "Kazakhstan", "Kyrgyz Republic", "Latvia", "Lithuania", "Moldova", "North Macedonia", "Russia", "Slovenia", "Tajikistan", "Turkmenistan", "Ukraine", "Uzbekistan"]
df_popfood_v4 = df_popfood_v3[~df_popfood_v3['country'].isin(countries_fillin_data)]

# confirm countries have been dropped with Croatia as example
df_popfood_v4.query("country == 'Croatia'")

,country,year,population,foodsupply


In [326]:
# drop Oman 
finaldrop = ["Oman"]
df_popfood_v5 = df_popfood_v4[~df_popfood_v4['country'].isin(finaldrop)]

# confirm dropped
df_popfood_v5.query("country =='Oman'")

,country,year,population,foodsupply


In [327]:
df_popfood_v5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7473 entries, 0 to 8903
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     7473 non-null   object 
 1   year        7473 non-null   object 
 2   population  7473 non-null   int64  
 3   foodsupply  7473 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 291.9+ KB


In [362]:
df_popfood_v5.isnull().sum()

country       0
year          0
population    0
foodsupply    0
dtype: int64

The information provided above indicates that columns show appropriate data type for analysis. In addition, there are 7473 entries with no null values.   

In [328]:
# save clean data for population and food supply analysis
df_popfood_v5.to_csv('df_popfood_clean.csv', index=False)

#### Cleaning datasets for further analysis

The cleaned dataset above will be further refined by:
- filling in null values for the countries identified previously. 
- dropping columns so only 1990-2013 range are included.
- join the 'suicide.csv' dataset
- join the 'internet.csv' dataset
- investigate possible null values

In [367]:
# load dataset
df_popfood_final_v2.head()

,country,year,population,foodsupply
0,Afghanistan,1961,9170000,3000.0
1,Afghanistan,1962,9350000,2920.0
2,Afghanistan,1963,9540000,2700.0
3,Afghanistan,1964,9740000,2950.0
4,Afghanistan,1965,9960000,2960.0


Focus on continuing to clean data for later analysis

In [375]:
def null_values(df, countrylist_null):
    """
    Arg:
    
    Return:
    """
    
    for x in countrylist_null:
        value_92 = df[(df["country"] == x) & (df["year"] == "1992") ]["foodsupply"]
        value_93 = df[(df["country"] == x) & (df["year"] == "1993") ]["foodsupply"]
        value_94 = df[(df["country"] == x) & (df["year"] == "1994") ]["foodsupply"]

        rate_92_93 = (value_93 - value_92) / value_92
        rate_93_94 = (value_94 - value_93) / value_93
        rate = (sum(rate_92_93, rate_93_94))/2
        
        value_91 = -1 * rate_92_93 * value_92
        value_90 = -1 * rate_93_94 * value_91
        
        df[(df["country"] == x) & (df["year"] == "1991")].fillna(value_91)
        df[(df["country"] == x) & (df["year"] == "1990")].fillna(value_90)
        
    return df


In [377]:
#Values will be estimated by seeing the 
countries_fillin_data = ["Armenia", "Azerbaijan", "Belarus", "Bosnia and Herzegovina", "Croatia", "Estonia", "Georgia", "Kazakhstan", "Kyrgyz Republic", "Latvia", "Lithuania", "Moldova", "North Macedonia", "Russia", "Slovenia", "Tajikistan", "Turkmenistan", "Ukraine", "Uzbekistan"]

# use to fill in data
dfnull = null_values(df_popfood_final_v2, countries_fillin_data)

# confirm success
dfnull.query('country == "Belarus"')

,country,year,population,foodsupply
689,Belarus,1961,8200000,NaN
690,Belarus,1962,8280000,NaN
691,Belarus,1963,8350000,NaN
692,Belarus,1964,8430000,NaN
693,Belarus,1965,8510000,NaN
694,Belarus,1966,8590000,NaN
695,Belarus,1967,8670000,NaN
696,Belarus,1968,8750000,NaN
697,Belarus,1969,8840000,NaN
698,Belarus,1970,8910000,NaN


In [ ]:
# melt suicide and internet dataframe
df_tomelt = [df_suicide, df_internet]
columns_new = ["suicide", "internet"]

df_foodint_temp = df_melt(df_tomelt, column_new)

# join dataframes
df_foodint = reduce(lambda left,right: pd.merge(left,right, on=["country", "year"], how = 'left'), df_foodint_temp)

df_foodint.head()

In [ ]:
# drop years outside of 1990 - 2013 range
df_pf = df_popfood_final_v2
df_pf_drop = df_pf.drop(df_pf[(df_pf.year < 1990) & (df_pf.year > 2013)].index)

In [ ]:
# join with population dataframe
df_join = [df_foodint, df_pf_drop]
df_main = reduce(lambda left,right: pd.merge left,right, on=['country', 'year'], how = 'left'), df_join)

# confirm merge
df_main.head()

In [ ]:
.info() 
make sure everything is in the correct type

<a id='explore'></a>
## Exploratory Data Analysis

<a id='Q1'></a>
### Q1. How has the consumption of food per person (per day) changed over time throughout the world?

In [ ]:
df_popfood_final = pd.read_csv('df_popfood_clean.csv')
df_popfood_final.head()

In [ ]:
# line plot showing consumption of food per person over time


<a id='Q2'></a>
### Q2. How has the world's population changed over the same time period above?

<a id='Q3'></a>
### Q3

<a id='Q4'></a>
### Q4

<a id='summary'></a>
## Summary & Conclusions